In [1]:
!pip install --user xlrd

In [2]:
# Setup constants if any




In [2]:
import pandas as pd
from io import BytesIO
import requests
import json
import xlrd 

from pyspark.sql.functions import *
from pyspark.sql.types import *

from datetime import datetime
from dateutil.parser import parse

from ingest.Connectors import Connectors

In [3]:
# The code was removed by DSX for sharing.

In [4]:
# The code was removed by DSX for sharing.

In [29]:

balEquityByCurrDF = pd.read_csv(getFileFromObjectStorage('MizuhoPOC', 'FCM-NE_Balance_Equity_by_Currency_20170731.csv'))

# Drop rows & columns with all 'NaN' values, axis 0 is for row
# balEquityByCurrDF = balEquityByCurrDF.fillna('')

balEquityByCurrDF.head(20)



balEquityByCurrRenamedDF = balEquityByCurrDF.rename(index=str, 
                                                columns={"BALANCETYPE": "BAL_TYPE", 
                                                         "SortLevel": "SORT_LEVEL",                                                         
                                                         "Company": "COMPANY",
                                                         "firm": "FIRM",
                                                         "Office": "OFFICE",
                                                         "Account": "ACCT_NUM",                                                          
                                                         "Salesman": "SALESMAN", 
                                                         "LBDAY": "AS_OF_DATE",
                                                         "ACCOUNTNAME": "FUND_NAME",
                                                         "GROUPACCTNUM": "GROUP_ACCT_NUM",
                                                         "GROUPACCTNAM": "GROUP_FUND_NAME",
                                                         "ACCTTYPE":"ACCT_TYPE",
                                                         "CURR": "CURRENCY",
                                                         "BALANCE": "BALANCE",
                                                         "FUTINITMARGIN": "FUT_INIT_MARGIN",
                                                         "FUTMAINTMARGIN":"FUT_MAINT_MARGIN",
                                                         "SECLONGMKVAL": "SEC_LONG_MKVAL",                                                         
                                                         "SECSHORTMKVAL": "SEC_SHORT_MKVAL",
                                                         "OPTLONGMKVAL": "OPT_LONG_MKVAL",
                                                         "OPTSHORTMKVAL": "OPT_SHORT_MKVAL",
                                                         "OPTLONGOTE": "OPT_LONG_OTE",
                                                         "OPTSHORTOTE":"OPT_SHORT_OTE",
                                                         "NETLIQ":"NET_LIQ",
                                                         "MARGINEXCESS":"MARGIN_EXCESS",
                                                         "MARGINCOLVAL":"MARGIN_COL_VAL",
                                                         "FUTOTE":"FUT_OTE",
                                                         "TOTALEQUITY":"TOTAL_EQUITY",
                                                         "WITHDRAWABLE": "WITHDRAWABLE",
                                                         "TOTCALLS":"TOT_CALLS",
                                                         "ACCTCURR":"ACCT_CURR",
                                                         "MASTCURR":"MAST_CURR",
                                                         "CONVACCT":"CONV_ACCT",
                                                         "CONVMAST":"CONV_MAST",
                                                         "RATEFAILS":"RATE_FAILS"
                                                        })

# Convert the datetimeindex object to date
balEquityByCurrRenamedDF['AS_OF_DATE'] = pd.DatetimeIndex(balEquityByCurrRenamedDF['AS_OF_DATE']).date

balEquityByCurrRenamedDF[['BALANCE', 'FUT_INIT_MARGIN', 'FUT_MAINT_MARGIN','SEC_LONG_MKVAL','SEC_SHORT_MKVAL', 'OPT_LONG_MKVAL', 'OPT_SHORT_MKVAL', 'OPT_LONG_OTE', 'OPT_SHORT_OTE', 'NET_LIQ', 'MARGIN_EXCESS', 'MARGIN_COL_VAL', 'FUT_OTE', 'TOTAL_EQUITY', 'WITHDRAWABLE', 'TOT_CALLS', 'CONV_ACCT', 'CONV_MAST']] = \
balEquityByCurrRenamedDF[['BALANCE', 'FUT_INIT_MARGIN', 'FUT_MAINT_MARGIN','SEC_LONG_MKVAL','SEC_SHORT_MKVAL', 'OPT_LONG_MKVAL', 'OPT_SHORT_MKVAL', 'OPT_LONG_OTE', 'OPT_SHORT_OTE', 'NET_LIQ', 'MARGIN_EXCESS', 'MARGIN_COL_VAL', 'FUT_OTE', 'TOTAL_EQUITY', 'WITHDRAWABLE', 'TOT_CALLS', 'CONV_ACCT', 'CONV_MAST']].astype(float)

# balEquityByCurrRenamedDF.head(20)
print balEquityByCurrRenamedDF.dtypes

BAL_TYPE             object
SORT_LEVEL           object
COMPANY              object
FIRM                 object
OFFICE               object
ACCT_NUM             object
SALESMAN             object
AS_OF_DATE           object
FUND_NAME            object
GROUP_ACCT_NUM       object
GROUP_FUND_NAME      object
ACCT_TYPE            object
CURRENCY             object
BALANCE             float64
FUT_INIT_MARGIN     float64
FUT_MAINT_MARGIN    float64
SEC_LONG_MKVAL      float64
SEC_SHORT_MKVAL     float64
OPT_LONG_MKVAL      float64
OPT_SHORT_MKVAL     float64
OPT_LONG_OTE        float64
OPT_SHORT_OTE       float64
NET_LIQ             float64
MARGIN_EXCESS       float64
MARGIN_COL_VAL      float64
FUT_OTE             float64
TOTAL_EQUITY        float64
WITHDRAWABLE        float64
TOT_CALLS           float64
ACCT_CURR            object
MAST_CURR            object
CONV_ACCT           float64
CONV_MAST           float64
RATE_FAILS          float64
dtype: object


In [49]:
spark = SparkSession.builder.getOrCreate()  

def build_schema():
    """Build and return a schema to use for the sample data."""
    schema = StructType(
        [
            StructField("BAL_TYPE", StringType(), True),
            StructField("SORT_LEVEL", StringType(), True),
            StructField("COMPANY", StringType(), True),
            StructField("FIRM", StringType(), True),            
            StructField("OFFICE", StringType(), True),
            StructField("ACCT_NUM", StringType(), False),
            StructField("SALESMAN", StringType(), True),
            StructField("AS_OF_DATE", DateType(), False),
            StructField("FUND_NAME", StringType(), False),
            StructField("GROUP_ACCT_NUM", StringType(), True),
            StructField("GROUP_FUND_NAME", StringType(), True),
            StructField("ACCT_TYPE", StringType(), True),            
            StructField("CURRENCY", StringType(), True),                        
            StructField("BALANCE", DoubleType(), True),   
            StructField("FUT_INIT_MARGIN", DoubleType(), True),   
            StructField("FUT_MAINT_MARGIN", DoubleType(), True),   
            StructField("SEC_LONG_MKVAL", DoubleType(), True),   
            StructField("SEC_SHORT_MKVAL", DoubleType(), True),               
            StructField("OPT_LONG_MKVAL", DoubleType(), True),   
            StructField("OPT_SHORT_MKVAL", DoubleType(), True),               
            StructField("OPT_LONG_OTE", DoubleType(), True),               
            StructField("OPT_SHORT_OTE", DoubleType(), True),   
            StructField("NET_LIQ", DoubleType(), True),   
            StructField("MARGIN_EXCESS", DoubleType(), True),               
            StructField("MARGIN_COL_VAL", DoubleType(), True),               
            StructField("FUT_OTE", DoubleType(), True),     
            StructField("TOTAL_EQUITY", DoubleType(), True), 
            StructField("WITHDRAWABLE", DoubleType(), True), 
            StructField("TOT_CALLS", DoubleType(), True),             
            StructField("ACCT_CURR", StringType(), True),
            StructField("MAST_CURR", StringType(), True),
            StructField("CONV_ACCT", DoubleType(), True),     
            StructField("CONV_MAST", DoubleType(), True), 
            StructField("RATE_FAILS", DoubleType(), True)
        ]
    )
    return schema


balEquityByCurrSparkDF = spark.createDataFrame(balEquityByCurrRenamedDF, schema=build_schema())

balEquityByCurrSparkDF.printSchema()

balEquityByCurrSparkDF.show()




root
 |-- BAL_TYPE: string (nullable = true)
 |-- SORT_LEVEL: string (nullable = true)
 |-- COMPANY: string (nullable = true)
 |-- FIRM: string (nullable = true)
 |-- OFFICE: string (nullable = true)
 |-- ACCT_NUM: string (nullable = false)
 |-- SALESMAN: string (nullable = true)
 |-- AS_OF_DATE: date (nullable = false)
 |-- FUND_NAME: string (nullable = false)
 |-- GROUP_ACCT_NUM: string (nullable = true)
 |-- GROUP_FUND_NAME: string (nullable = true)
 |-- ACCT_TYPE: string (nullable = true)
 |-- CURRENCY: string (nullable = true)
 |-- BALANCE: double (nullable = true)
 |-- FUT_INIT_MARGIN: double (nullable = true)
 |-- FUT_MAINT_MARGIN: double (nullable = true)
 |-- SEC_LONG_MKVAL: double (nullable = true)
 |-- SEC_SHORT_MKVAL: double (nullable = true)
 |-- OPT_LONG_MKVAL: double (nullable = true)
 |-- OPT_SHORT_MKVAL: double (nullable = true)
 |-- OPT_LONG_OTE: double (nullable = true)
 |-- OPT_SHORT_OTE: double (nullable = true)
 |-- NET_LIQ: double (nullable = true)
 |-- MARGIN_EX

In [34]:
dashDBloadOptions = { 
                    Connectors.DASHDB.HOST              : dashCredentials["host"],
                    Connectors.DASHDB.DATABASE          : dashCredentials["db"],
                    Connectors.DASHDB.USERNAME          : dashCredentials["username"],
                    Connectors.DASHDB.PASSWORD          : dashCredentials["password"],
                    Connectors.DASHDB.SOURCE_TABLE_NAME : dashCredentials["REF_FUND_MAPPING_TABLE"],
}

refFundMappingDF = sqlContext.read.format("com.ibm.spark.discover").options(**dashDBloadOptions).load()
refFundMappingDF.printSchema()
refFundMappingDF.show(1)

root
 |-- ALADDIN_ID: string (nullable = true)
 |-- FUND_NAME: string (nullable = true)
 |-- FUTURES_ACCT_ID: string (nullable = true)
 |-- BTIG_ID: string (nullable = true)
 |-- FUND_ID: string (nullable = true)
 |-- NEWEDGE_ID: string (nullable = true)
 |-- BASE: string (nullable = true)
 |-- BARC_ID: string (nullable = true)

+----------+------------------+---------------+-------+---------+----------+----+-------+
|ALADDIN_ID|         FUND_NAME|FUTURES_ACCT_ID|BTIG_ID|  FUND_ID|NEWEDGE_ID|BASE|BARC_ID|
+----------+------------------+---------------+-------+---------+----------+----+-------+
|     I-CJF|CRYSTAL JAPAN FUND|          C6500|       |I-ASIAPAC|     72590|  JY|  4902C|
+----------+------------------+---------------+-------+---------+----------+----+-------+
only showing top 1 row



In [53]:
# balEquityByCurrSparkDF.printSchema()

balEquityByCurrJoinSparkDF = balEquityByCurrSparkDF.join(refFundMappingDF, 
                                               balEquityByCurrSparkDF.FUND_NAME == refFundMappingDF.FUND_NAME, "inner")\
                                        .select(
                                            refFundMappingDF.ALADDIN_ID.alias("FUND_ID"),
                                            balEquityByCurrSparkDF.BAL_TYPE,
                                            balEquityByCurrSparkDF.SORT_LEVEL,
                                            balEquityByCurrSparkDF.COMPANY,
                                            balEquityByCurrSparkDF.FIRM,
                                            balEquityByCurrSparkDF.OFFICE,
                                            balEquityByCurrSparkDF.ACCT_NUM,
                                            balEquityByCurrSparkDF.SALESMAN,  
                                            balEquityByCurrSparkDF.AS_OF_DATE,
                                            balEquityByCurrSparkDF.FUND_NAME,
                                            balEquityByCurrSparkDF.GROUP_ACCT_NUM,
                                            balEquityByCurrSparkDF.GROUP_FUND_NAME,
                                            balEquityByCurrSparkDF.ACCT_TYPE,
                                            balEquityByCurrSparkDF.CURRENCY,
                                            balEquityByCurrSparkDF.BALANCE,
                                            balEquityByCurrSparkDF.FUT_INIT_MARGIN,
                                            balEquityByCurrSparkDF.FUT_MAINT_MARGIN,
                                            balEquityByCurrSparkDF.SEC_LONG_MKVAL,
                                            balEquityByCurrSparkDF.SEC_SHORT_MKVAL,
                                            balEquityByCurrSparkDF.OPT_LONG_MKVAL,
                                            balEquityByCurrSparkDF.OPT_SHORT_MKVAL,
                                            balEquityByCurrSparkDF.OPT_LONG_OTE,
                                            balEquityByCurrSparkDF.OPT_SHORT_OTE,
                                            balEquityByCurrSparkDF.NET_LIQ,
                                            balEquityByCurrSparkDF.MARGIN_EXCESS,
                                            balEquityByCurrSparkDF.MARGIN_COL_VAL,
                                            balEquityByCurrSparkDF.FUT_OTE,
                                            balEquityByCurrSparkDF.TOTAL_EQUITY,
                                            balEquityByCurrSparkDF.WITHDRAWABLE,
                                            balEquityByCurrSparkDF.TOT_CALLS,
                                            balEquityByCurrSparkDF.ACCT_CURR,
                                            balEquityByCurrSparkDF.MAST_CURR,
                                            balEquityByCurrSparkDF.CONV_ACCT,
                                            balEquityByCurrSparkDF.CONV_MAST,
                                            balEquityByCurrSparkDF.RATE_FAILS
                                            )

balEquityByCurrJoinSparkDF.show(1)



+-------+--------+----------+-------+----+------+--------+--------+----------+------------------+--------------+------------------+---------+--------+-------+---------------+----------------+--------------+---------------+--------------+---------------+------------+-------------+-------+-------------+--------------+-------+------------+------------+---------+---------+---------+-----------+---------+----------+
|FUND_ID|BAL_TYPE|SORT_LEVEL|COMPANY|FIRM|OFFICE|ACCT_NUM|SALESMAN|AS_OF_DATE|         FUND_NAME|GROUP_ACCT_NUM|   GROUP_FUND_NAME|ACCT_TYPE|CURRENCY|BALANCE|FUT_INIT_MARGIN|FUT_MAINT_MARGIN|SEC_LONG_MKVAL|SEC_SHORT_MKVAL|OPT_LONG_MKVAL|OPT_SHORT_MKVAL|OPT_LONG_OTE|OPT_SHORT_OTE|NET_LIQ|MARGIN_EXCESS|MARGIN_COL_VAL|FUT_OTE|TOTAL_EQUITY|WITHDRAWABLE|TOT_CALLS|ACCT_CURR|MAST_CURR|  CONV_ACCT|CONV_MAST|RATE_FAILS|
+-------+--------+----------+-------+----+------+--------+--------+----------+------------------+--------------+------------------+---------+--------+-------+------------

In [55]:
# Connection to Dash DB for writing the data
dashdbsaveoption = {
                     Connectors.DASHDB.HOST              : dashCredentials["host"],
                     Connectors.DASHDB.DATABASE          : dashCredentials["db"],
                     Connectors.DASHDB.USERNAME          : dashCredentials["username"],
                     Connectors.DASHDB.PASSWORD          : dashCredentials["password"],
                     Connectors.DASHDB.TARGET_TABLE_NAME : dashCredentials["tableName"],
                     Connectors.DASHDB.TARGET_WRITE_MODE : 'merge' 
}

balEquityByCurrJoinSparkDF.printSchema()

saveDashDBDF = balEquityByCurrJoinSparkDF.write.format("com.ibm.spark.discover").options(**dashdbsaveoption).save()

root
 |-- FUND_ID: string (nullable = true)
 |-- BAL_TYPE: string (nullable = true)
 |-- SORT_LEVEL: string (nullable = true)
 |-- COMPANY: string (nullable = true)
 |-- FIRM: string (nullable = true)
 |-- OFFICE: string (nullable = true)
 |-- ACCT_NUM: string (nullable = false)
 |-- SALESMAN: string (nullable = true)
 |-- AS_OF_DATE: date (nullable = false)
 |-- FUND_NAME: string (nullable = false)
 |-- GROUP_ACCT_NUM: string (nullable = true)
 |-- GROUP_FUND_NAME: string (nullable = true)
 |-- ACCT_TYPE: string (nullable = true)
 |-- CURRENCY: string (nullable = true)
 |-- BALANCE: double (nullable = true)
 |-- FUT_INIT_MARGIN: double (nullable = true)
 |-- FUT_MAINT_MARGIN: double (nullable = true)
 |-- SEC_LONG_MKVAL: double (nullable = true)
 |-- SEC_SHORT_MKVAL: double (nullable = true)
 |-- OPT_LONG_MKVAL: double (nullable = true)
 |-- OPT_SHORT_MKVAL: double (nullable = true)
 |-- OPT_LONG_OTE: double (nullable = true)
 |-- OPT_SHORT_OTE: double (nullable = true)
 |-- NET_LIQ: 